In [1]:
import requests # for making standard html requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup # magical tool for parsing html data
import json # for parsing data
import pandas as pd
import csv
import random
import time
import glob

In [2]:
# Build list of user agents to avoid being blocked.
user_agents = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
               "Mozilla/5.0"]
containers = []

# Pull 10 pages of listings for list of suburbs in Cork city area.
for location in ['douglas','ballincollig','carrigaline','bishopstown','glasheen','blackrock','rochestown',
    'lehenaghmore','ballinlough','ballintemple','wilton','model-farm-road','turners-cross']:
#for location in ['wilton']:
    for offset in range(0,60,20): # 20 listings per page
        user_agent = random.sample(user_agents,1)
        url = "http://www.daft.ie/cork-city/houses-for-sale/" + location + "/?offset=" + str(offset)
        req = Request(url , headers={'User-Agent': str(user_agent)})
        webpage = urlopen(req).read()
        page_soup = soup(webpage, "html.parser")
        containers += page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--premium")
        containers += page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--standard")
        time.sleep(5) # Wait 5 seconds between hitting site
        print(location,offset, len(containers))

douglas 0 18
douglas 20 38
douglas 40 58
ballincollig 0 74
ballincollig 20 94
ballincollig 40 94
carrigaline 0 111
carrigaline 20 131
carrigaline 40 140
bishopstown 0 159
bishopstown 20 167
bishopstown 40 167
glasheen 0 182
glasheen 20 182
glasheen 40 182
blackrock 0 201
blackrock 20 206
blackrock 40 206
rochestown 0 226
rochestown 20 227
rochestown 40 227
lehenaghmore 0 235
lehenaghmore 20 235
lehenaghmore 40 235
ballinlough 0 245
ballinlough 20 245
ballinlough 40 245
ballintemple 0 250
ballintemple 20 250
ballintemple 40 250
wilton 0 256
wilton 20 256
wilton 40 256
model-farm-road 0 267
model-farm-road 20 267
model-farm-road 40 267
turners-cross 0 272
turners-cross 20 272
turners-cross 40 272


In [5]:
# Check all previous scrapes. If there is a new listing in this run then pull additional details from listing page
df = pd.concat([pd.read_csv(f) for f in glob.glob('cork_property_prices*.csv')], ignore_index = True)
df = df.drop_duplicates(subset='listing_id', keep='last') # Dedup on listing_id and keep first record.
listings = set(df['listing_id'])
listings

{1199531,
 1344639,
 1498813,
 1628232,
 1644605,
 1699244,
 1790918,
 1859049,
 1878287,
 1927033,
 1938210,
 1949004,
 1983542,
 1996692,
 2024629,
 2069030,
 2078980,
 2085468,
 2086148,
 2097675,
 2098774,
 2103851,
 2110952,
 2112502,
 2112503,
 2114906,
 2117961,
 2125026,
 2130914,
 2139860,
 2141109,
 2143476,
 2154356,
 2159800,
 2159801,
 2160128,
 2162034,
 2162977,
 2164698,
 2165789,
 2169216,
 2173346,
 2173352,
 2175903,
 2189745,
 2189830,
 2200534,
 2202506,
 2215743,
 2217312,
 2223530,
 2225567,
 2227935,
 2233530,
 2235605,
 2235754,
 2236215,
 2237405,
 2255031,
 2255069,
 2260856,
 2262649,
 2266945,
 2278682,
 2288134,
 2291989,
 2300051,
 2306077,
 2308866,
 2314593,
 2316012,
 2318639,
 2321032,
 2321596,
 2326179,
 2340335,
 2340566,
 2341410,
 2341411,
 2342163,
 2343206,
 2355909,
 2358241,
 2368798,
 2371157,
 2375737,
 2378977,
 2379592,
 2379662,
 2379811,
 2383050,
 2384507,
 2384808,
 2388509,
 2388627,
 2390452,
 2390689,
 2392062,
 2396110,
 2396460,


In [6]:
# For each kind of listing, create a list of containers.
# containers = page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--premium")
# containers += page_soup.findAll("div","AdCard__adCardContainer AdCard__adCardContainer--standard")

# Check all previous scrapes. If there is a new listing in this run then pull additional details from listing page
df = pd.concat([pd.read_csv(f) for f in glob.glob('cork_property_prices*.csv')], ignore_index = True)
df = df.drop_duplicates(subset='listing_id', keep='last') # Dedup on listing_id and keep first record.
listings = set(df['listing_id'])
#listings = [0,0,0,]

# Write data to an array for output to csv later
rows = []

for idx, container in enumerate(containers):
    listing = {}
    # Check if the listing is "valid", ads will not have a price section.
    if container.find('strong', "PropertyInformationCommonStyles__costAmountCopy") is None:
        print("Ad Detected!")
    else: 
        listing['listing_id'] = container.find('a',"PropertyInformationCommonStyles__addressCopy--link")["href"].split("-")[-1][:-1]
        print(listing['listing_id'])
        listing['url'] = container.find('a',"PropertyInformationCommonStyles__addressCopy--link")["href"]
        listing['location'] = container.find('a',"PropertyInformationCommonStyles__propertyPrice--link")["href"].split("/")[3]
        listing['address'] = container.find('a',"PropertyInformationCommonStyles__addressCopy--link").getText().replace(",","|")
        listing['beds'] = container.find('div', "QuickPropertyDetails__iconCopy").getText()
        listing['bathrooms'] = container.find('div', "QuickPropertyDetails__iconCopy--WithBorder").getText()
        listing['cost'] = container.find('strong', "PropertyInformationCommonStyles__costAmountCopy").getText().replace("€","")
        listing['num_pics'] = container.find('span', "PropertyImage__picturesAmountCopy").getText()
        listing['property_type'] = container.find('div', "QuickPropertyDetails__propertyType").getText().replace("\n","").strip()
        try:
            listing['ber'] = container.find("img",{"class":"PropertyImage__berImage"}).get_attribute_list("src")[0][-6:-4]
        except:
            listing['ber'] = 'NA'
        
        if int(listing['listing_id']) not in listings:
            print("New listing found:")
            url = "http://www.daft.ie/" + listing['url']
            print(url)
            req = Request(url , headers={'User-Agent': str(user_agent)})
            webpage = urlopen(req).read()
            page_soup = soup(webpage, "html.parser")
            try:
                listing['eircode'] = page_soup.find('div', "PropertyMainInformation__eircode").getText().split(":")[1].strip().replace("\xa0"," ")
            except AttributeError as eir_exc:
                print("error pulling eircode:", eir_exc)
                listing['eircode'] = ""
            propertyOverviewDetails = page_soup.find('div', "PropertyOverview__propertyOverviewDetails").getText()
            if "Overall Floor Area" in propertyOverviewDetails:
                listing['floor_area'] = propertyOverviewDetails.split("Overall Floor Area:")[1].strip().split(" ")[0]
            else:
                listing['floor_area'] = ""
            time.sleep(5)
            rows.append(list(listing.values())) # Add listing to rows list for output.
        #else:
        #    # Old listing
        #    listing['eircode'] = ""
        #    listing['floor_area'] = 0 

2162034
2425053
2578874
2549821
2549894
2255069
2217312
Ad Detected!
2603357
New listing found:
http://www.daft.ie//cork/houses-for-sale/douglas/24-ardfallen-road-douglas-douglas-cork-2603357/
2602802
2600677
2593331
2588167
2583721
2580799
2563871
2562363
2562256
2557773
2553991
2552907
2549333
2549329
2548380
2255031
2540071
2535370
2154356
2530169
2069030
2125026
2525283
2499444
2495035
2451964
2445459
2439236
2434271
2425069
2408600
2417704
2405628
2396711
2392062
2379811
2375737
2141109
2341411
2321596
2314593
2306077
2200534
2159800
1996692
2159801
1790918
1983542
1699244
2590281
2588094
2566059
2560238
2414340
2260856
Ad Detected!
Ad Detected!
2605409
New listing found:
http://www.daft.ie//cork/houses-for-sale/ballincollig/182-the-willows-ballincollig-cork-2605409/
2602798
2600311
2593578
2589743
2579359
2578603
2571509
2562022
2551110
2489588
2535638
2173346
2384808
2441777
2512113
2467926
2422436
2407581
2405604
2396460
2379592
2371157
2308866
2227935
2225567
2169216
2117961
2

In [7]:
len(rows)

16

In [52]:
# Check all previous scrapes. If there is a new listing in this run then pull additional details from the page of that listing
df = pd.concat([pd.read_csv(f) for f in glob.glob('cork_property_prices*.csv')], ignore_index = True)
df = df.drop_duplicates(subset='listing_id', keep='last') # Dedup on listing_id and keep first record.
listings = set(df['listing_id'])

for row in rows: 
    if int(row[0]) not in listings:
        url = "http://www.daft.ie/" + row[1]
        print(url)
        req = Request(url , headers={'User-Agent': str(user_agent)})
        webpage = urlopen(req).read()
        page_soup = soup(webpage, "html.parser")
        eircode = page_soup.find('div', "PropertyMainInformation__eircode").getText().split(":")[1].strip()
        propertyOverviewDetails = page_soup.find('div', "PropertyOverview__propertyOverviewDetails").getText()
        if "Overall Floor Area" in propertyOverviewDetails:
            floor_area = propertyOverviewDetails.split("Overall Floor Area:")[1].strip().split(" ")[0]
            print(floor_area)
        time.sleep(5)
        floor = float(floor_area)
# Get eircode <div class="PropertyMainInformation__eircode">
# Get floor area <span class="PropertyOverview__floorArea">Overall Floor Area:</span>


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


http://www.daft.ie//cork/houses-for-sale/douglas/1-well-road-homes-churchyard-lane-douglas-cork-2600677/
74.3
http://www.daft.ie//cork/houses-for-sale/ballincollig/46-tuairin-glas-greenfield-ballincollig-cork-2600311/
http://www.daft.ie//cork/houses-for-sale/bishopstown/maryville-48-rossa-avenue-bishopstown-cork-2419284/
158.18
http://www.daft.ie//cork/houses-for-sale/cork-city/54-wilton-road-cork-city-cork-2544464/
109


In [8]:

# Output to flat file for modeling 
#filename = "cork_property_prices.csv"
filename = "cork_property_prices_" + str(round(time.time())) + ".csv"

# field names  
fields = ['listing_id', 'url', 'location', 'address', 'beds', 'bathrooms', 'cost', 'num_pics', 'property_type', 'ber','eircode','floor_area'] 

# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(fields)  
        
    # writing the data rows  
    csvwriter.writerows(rows) 